In [7]:
import math
# https://hughchen.github.io/its_blog/index.html#define_tree_samples 여기 동적계획법 shap 계산 예제

In [8]:
class Testtree():
    def __init__(
        self,
        is_terminal: bool,
        best_feature: int,
        best_feature_val: int,
        val= None,
        
    ):
        self.is_terminal = is_terminal
        self.best_feature = best_feature
        self.best_feature_val = best_feature_val
        self.val = val

        self.l_tree = None
        self.r_tree = None

        self.posval = None
        self.negval = None

def weight(sc,nc):
    pw = 0
    nw = 0
    if sc == 0:
        if nc != 0:
            nw = -1*(math.factorial(sc) * math.factorial(nc - sc - 1))/math.factorial(nc)
        return pw,nw
    if sc !=0:
        pw = (math.factorial(sc-1) * math.factorial(nc - (sc -1) - 1))/math.factorial(nc)
    if sc != nc:
        nw = -1*(math.factorial(sc) * math.factorial(nc - sc - 1))/math.factorial(nc)

    return pw, nw

def treeshap_dynamic_test(x_foreground, x_background, tree):

    def recurse(node_tree, n_c, s_c, foreground_check, background_check):
        

        if node_tree.is_terminal:
            pw, nw = weight(s_c,n_c)
            val_sum = node_tree.val   #여기를 만약에 sum으로하면 리프가 갖는 데이터셋 개수에 따라서 값차이가 생기므로 평균으로 처리해야함
            
            node_tree.posval = val_sum * pw
            node_tree.negval = val_sum * nw
            return val_sum * pw, val_sum * nw
        
        

        xf_val = x_foreground[node_tree.best_feature - 1]
        xb_val = x_background[node_tree.best_feature - 1]

        xf_child = node_tree.l_tree if xf_val < node_tree.best_feature_val else node_tree.r_tree
        xb_child = node_tree.l_tree if xb_val < node_tree.best_feature_val else node_tree.r_tree
            
        if foreground_check[node_tree.best_feature -1 ] > 0:
            return recurse(xf_child,n_c,s_c, foreground_check, background_check)
        
        if background_check[node_tree.best_feature -1 ] > 0:
            return recurse(xb_child,n_c,s_c, foreground_check, background_check)

        if xf_child == xb_child:
            return recurse(xb_child,n_c,s_c, foreground_check, background_check)

        if xf_child != xb_child:
            # foreground search
            foreground_check[node_tree.best_feature -1 ] += 1
            posf, negf = recurse(xf_child,n_c + 1,s_c + 1, foreground_check, background_check)
            foreground_check[node_tree.best_feature -1 ] -= 1


            # background search
            background_check[node_tree.best_feature -1 ] += 1
            posb, negb = recurse(xb_child,n_c + 1,s_c, foreground_check, background_check)
            background_check[node_tree.best_feature -1 ] -= 1

            # add phi
            phi[node_tree.best_feature - 1] += (posf + negb)
            
            node_tree.posval = posf + posb
            node_tree.negval = negf + negb
            return posf + posb, negf + negb
    
    recurse(tree, 0, 0, feature_forecheck, feature_backcheck)

In [9]:
tree1 = Testtree(False,1,5)
tree2 = Testtree(False,2,-5)
tree3 = Testtree(False,2,5)
tree4 = Testtree(False,1,5)
tree5 = Testtree(False,3,5)
tree6 = Testtree(False,3,-5)
tree7 = Testtree(False,1,5)
tree8 = Testtree(True,1,5,val=1)
tree9 = Testtree(True,1,5,val=2)
tree10 = Testtree(True,1,5,val=3)
tree11 = Testtree(True,1,5,val=4)
tree12 = Testtree(True,1,5,val=5)
tree13 = Testtree(True,1,5,val=6)
tree14 = Testtree(True,1,5,val=7)
tree15 = Testtree(True,1,5,val=8)

tree1.l_tree = tree2
tree1.r_tree = tree3

tree2.l_tree = tree4
tree2.r_tree = tree5

tree3.l_tree = tree6
tree3.r_tree = tree7

tree4.l_tree = tree8
tree4.r_tree = tree9

tree5.l_tree = tree10
tree5.r_tree = tree11

tree6.l_tree = tree12
tree6.r_tree = tree13

tree7.l_tree = tree14
tree7.r_tree = tree15

xf = [0,0,10]
xb = [10,10,0]

phi = [0] * len(xf)
feature_forecheck = [0] * len(xf)
feature_backcheck = [0] * len(xf)
            


treeshap_dynamic_test(xf, xb, tree1)
print(phi)

[-3.5, -1.0, 0.5]


In [10]:
tree15.posval

0

In [11]:
tree15.negval

-4.0